In [1]:
import numpy as np
import pandas as pd
import os
import sys
sys.path.append('/home/ZwZ/cpc/mimic3-benchmarks/')
from mimic3benchmark.mimic3csv import *
from mimic3benchmark.preprocessing import add_hcup_ccs_2015_groups, make_phenotype_label_matrix
from mimic3benchmark.util import dataframe_from_csv

In [19]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
mimic_path ='/home/ZwZ/database/MIMIC'

## Read Table

In [3]:
patients=read_patients_table(mimic_path)
admins=read_admissions_table(mimic_path)
stays=read_icustays_table(mimic_path)

In [4]:
patients.head(2)

,SUBJECT_ID,GENDER,DOB,DOD
ROW_ID,,,,
234,249,F,2075-03-13,NaT
235,250,F,2164-12-27,2188-11-22


In [5]:
admins.head(2)

,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ETHNICITY,DIAGNOSIS
ROW_ID,,,,,,,
21,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,NaT,WHITE,BENZODIAZEPINE OVERDOSE
22,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,NaT,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...


## Filtering

In [7]:
stays = remove_icustays_with_transfers(stays)

# dimension inspection
stays.shape

(55830, 8)

In [8]:
stays = merge_on_subject_admission(stays,admins)
stays = merge_on_subject(stays, patients)

stays.shape # columns increased 

(55830, 16)

### further filtering patients 

In [9]:
# the code inside filter_admissions_on_nb_icustays
to_keep = stays.groupby('HADM_ID').count()[['ICUSTAY_ID']].reset_index()

In [10]:
to_keep

,HADM_ID,ICUSTAY_ID
0,100001,1
1,100003,1
2,100006,1
3,100009,1
4,100010,1
...,...,...
52829,199993,1
52830,199994,1
52831,199995,1
52832,199998,1


In [11]:
stays = filter_admissions_on_nb_icustays(stays)
stays.shape

(50186, 16)

In [12]:
stays = add_age_to_icustays(stays)                               # add age info
stays = add_inunit_mortality_to_icustays(stays)                  # die in ICU ? 0: alive , 1: dead
stays = add_inhospital_mortality_to_icustays(stays)              # die in hospital ? 0 : alive , 1: dead
stays = filter_icustays_on_age(stays)                            # screening patients older than 18

stays.shape

(42276, 20)

## Diagnose

In [14]:
diagnoses = read_icd_diagnoses_table(mimic_path)
# screen patients
diagnoses = filter_diagnoses_on_stays(diagnoses, stays)

count_icd_codes(diagnoses)

,ICD9_CODE,SHORT_TITLE,LONG_TITLE,COUNT
0,4019,Hypertension NOS,Unspecified essential hypertension,17343
1,4280,CHF NOS,"Congestive heart failure, unspecified",10601
2,41401,Crnry athrscl natve vssl,Coronary atherosclerosis of native coronary ar...,10345
3,42731,Atrial fibrillation,Atrial fibrillation,10313
4,25000,DMII wo cmp nt st uncntr,Diabetes mellitus without mention of complicat...,7486
...,...,...,...,...
6164,49301,Ext asthma w status asth,Extrinsic asthma with status asthmaticus,1
6165,73002,Ac osteomyelitis-up/arm,"Acute osteomyelitis, upper arm",1
6166,36801,Strabismic amblyopia,Strabismic amblyopia,1
6167,37952,Latent nystagmus,Latent nystagmus,1


In [18]:
diagnoses.groupby('ICD9_CODE')['ICUSTAY_ID'].count() 

ICD9_CODE
0030     2
0038     1
0039     1
0041     3
0048     1
        ..
V8821    3
V9010    1
V9039    1
V9081    3
V9089    1
Name: ICUSTAY_ID, Length: 6169, dtype: int64

In [21]:
?read_admissions_table

In [23]:
! ls

test.ipynb  utils.py  what_extract_subject_does.ipynb


In [ ]:
codes = diagnoses[['ICD9_CODE', 'SHORT_TITLE', 'LONG_TITLE']].drop_duplicates().set_index('ICD9_CODE')
codes['COUNT'] = diagnoses.groupby('ICD9_CODE')['ICUSTAY_ID'].count() 